## Hello World, AutoGen

In this notebook, we will implement a simple `Assistant` Agent which generates hello-world message to user using `gpt-4o-mini` model.

In [2]:
from typing import List
from typing import List, Optional
from autogen_core.base import MessageContext
from autogen_core.components import DefaultTopicId, RoutedAgent, default_subscription, message_handler
from autogen_core.components.models import (
    AssistantMessage,
    ChatCompletionClient,
    LLMMessage,
    SystemMessage,
    UserMessage,
)
from pydantic import BaseModel
from autogen_core.application import SingleThreadedAgentRuntime
from autogen_core.components.models import OpenAIChatCompletionClient

C:\Users\xiaoyuz\AppData\Local\Temp\ipykernel_91220\2560692345.py:14: FutureWarning: OpenAIChatCompletionClient moved to autogen_ext. Please import it from autogen_ext.modelsChatCompletionClient.
  from autogen_core.components.models import OpenAIChatCompletionClient


## Load .env

In [3]:
%load_ext dotenv
%dotenv .env

## Create a Message data class
The `Message` class is a data contract class that holds a text content.

In [4]:
class Message(BaseModel):
    text: str
    source: Optional[str] = None # indicate which agent the message is from

## Create AutoGen Assistant Agent

The assistant will be invoked when it receives a `Message` from agent runtime. Then it will use LLM to generate a response message and publish it back to the agent runtime. This is implemented in `handle_message` message handler.

### What is agent runtime?
Agent runtime is a fundation layer provided by AutoGen 0.4x framework that facilitates the communication between agents. Agent can publish messages to agent runtime, and subscribe to messages from agent runtime.

### How to create a message handler to process message from agent runtime?
In AutoGen, you create a message handler for an agent by decorating a method with `@message_handler` decorator. The method needs to have a `message` parameter with the type of the message that this handler wants to handle.

### How is a message handler invoked?
When an agent receives a message from agent runtime, the agent runtime will look for a message handler that can handle the message based on the message type. If a message handler is found, it will be invoked with the message as the parameter.

In [7]:
@default_subscription
class Assistant(RoutedAgent):
    def __init__(self, model_client: ChatCompletionClient) -> None:
        super().__init__("An assistant agent.")
        self._model_client = model_client
        self._chat_history: List[LLMMessage] = [
            SystemMessage(
                content="""
            You are a helpful AI assistant. You greet the user by saying 'Hello World <name>! How can I help you today?'.
            """,
            )
        ]

    @message_handler
    async def handle_message(self, message: Message, ctx: MessageContext) -> None:
        self._chat_history.append(UserMessage(content=message.text, source="user"))
        result = await self._model_client.create(self._chat_history)
        print(f"\n{'-'*80}\nAssistant:\n{result.content}")
        self._chat_history.append(AssistantMessage(content=result.content, source="assistant"))
        await self.publish_message(Message(text=result.content), DefaultTopicId())

## Create a single-threaded agent runtime and send a message to agent

### What is a single-threaded agent runtime?
A single-threaded agent runtime is a simple agent runtime that runs all agents in the memory, under same process and thread.

In [9]:
# Create an local embedded runtime.
import os;
runtime = SingleThreadedAgentRuntime()
await Assistant.register(
        runtime,
        "assistant",
        lambda: Assistant(
            OpenAIChatCompletionClient(
                model="gpt-4o-mini",
                # get api key from env:OPENAI_API_KEY
                api_key=os.environ.get("OPENAI_API_KEY"),
            )
        ),
    )

# Start the runtime and publish a message to the assistant.
runtime.start()
await runtime.publish_message(
    Message(text="Hello, I am Geeno"), DefaultTopicId()
)
await runtime.stop_when_idle()


--------------------------------------------------------------------------------
Assistant:
Hello World Geeno! How can I help you today?


## Conclusion

Congratulations! You have successfully created your first AutoGen Assistant Agent and sent a message to it.

## What we have learned so far
- Create a simple message data contract
- Create an assistant agent
- Add a message handler to the agent that processes messages from agent runtime
- Create a single-threaded agent runtime
- Send a message to the assistant agent